<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled231.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# S7 Companion: Curvature–Information Storm Analysis (outline)
# Requires: xarray, numpy, pandas, scipy, matplotlib, metpy, cartopy (optional)
import xarray as xr
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

# --------------------------
# 0) Inputs (edit these)
# --------------------------
ERA5_FILE = "era5_storm_cube.nc"      # 3D subset around the storm, hourly
BESTTRACK_CSV = "hurdat2_storm.csv"   # columns: time, lat, lon, vmax, mslp
LEVELS_FOR_PV = [850, 700, 500, 300]  # hPa
R_BINS_KM = np.array([0, 50, 100, 200])  # radial bins for aggregation

# --------------------------
# 1) Load data
# --------------------------
ds = xr.open_dataset(ERA5_FILE)  # expect variables: u, v, t, q, z (geopot), sst, lhf, shf, mslp
bt = pd.read_csv(BESTTRACK_CSV, parse_dates=["time"])

# Utility: haversine distance (km)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    lam1, lam2 = np.radians(lon1), np.radians(lon2)
    dphi = phi2 - phi1
    dlam = lam2 - lam1
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlam/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

# --------------------------
# 2) Derived fields
# --------------------------
# Moist static energy (MSE) and fluxes at e.g. 850 hPa (or column-integrated if available)
cp = 1004.0
Lv = 2.5e6
g0 = 9.81

# Ensure consistent dims (time, level, lat, lon)
# Example proxies:
u = ds["u"]    # m/s
v = ds["v"]    # m/s
T = ds["t"]    # K
q = ds["q"]    # kg/kg
Z = ds["z"]    # m^2/s^2 (geopotential); height ~ Z/g

# Choose a working level for fluxes (e.g., 850 hPa)
lev_sel = 850
ds_850 = ds.sel(level=lev_sel, method="nearest")
u850 = ds_850["u"]; v850 = ds_850["v"]; T850 = ds_850["t"]; q850 = ds_850["q"]; Z850 = ds_850["z"]/g0
MSE850 = cp*T850 + g0*Z850 + Lv*q850

# Horizontal flux of MSE
Fx = (ds_850["u"] * MSE850).transpose("time","lat","lon")
Fy = (ds_850["v"] * MSE850).transpose("time","lat","lon")

# Flux convergence (negative divergence = convergence)
# Simple centered differences on lat-lon (assumes regular grid)
lat = ds_850["lat"].values; lon = ds_850["lon"].values
deg2m_lat = np.pi/180.0*6371000.0
def grad_lon(F):
    dlam = np.deg2rad(np.gradient(lon))
    return np.gradient(F, axis=-1) / (dlam * np.cos(np.deg2rad(lat))[None,:,None]*6371000.0)
def grad_lat(F):
    dphi = np.deg2rad(np.gradient(lat))
    return np.gradient(F, axis=-2) / (dphi * 6371000.0)[None,:,None]

dFx_dx = grad_lon(Fx.values)
dFy_dy = grad_lat(Fy.values)
divF = dFx_dx + dFy_dy  # W/m^3 approx. (units depend on vertical choice)
convF_neg = np.minimum(divF, 0.0)      # keep convergent part
g2_field = convF_neg**2                 # proxy for g^2 (arb. scaling)

# Dephasing proxy: entropy production from surface fluxes (if available)
# Use ERA5 sensible (shf) + latent (lhf), distributed with a boundary-layer thickness H0
shf = ds["shf"] if "shf" in ds else 0*ds_850["t"]
lhf = ds["lhf"] if "lhf" in ds else 0*ds_850["t"]
Tskin = ds["sst"] if "sst" in ds else ds_850["t"]  # fallback
eps = 1e-6
gamma_phi_surf = (shf + lhf) / (Tskin + eps)  # W m^-2 K^-1

# Curvature proxy: PV gradient magnitude between 850–300 hPa
# Crude PV proxy: qg-PV ~ (f + zeta)/h  (use vorticity at 850–300 and thickness); or use ERA5 PV if present
if "pv" in ds:
    PV = ds["pv"].sel(level=slice(850,300))
else:
    # Minimal vorticity-based surrogate at 850 hPa
    # relative vorticity zeta = dv/dx - du/dy
    def ddx(F): return grad_lon(F)
    def ddy(F): return grad_lat(F)
    zeta = ddx(v850.values) - ddy(u850.values)
    PV = xr.DataArray(zeta, dims=("time","lat","lon"), coords=ds_850[["time","lat","lon"]])

PV_s = PV if "level" not in PV.dims else PV.mean("level")
PVx = grad_lon(PV_s.values)
PVy = grad_lat(PV_s.values)
kappa_field = np.sqrt(PVx**2 + PVy**2)  # magnitude of PV gradient

# --------------------------
# 3) Storm-centric aggregation
# --------------------------
times = pd.to_datetime(ds_850["time"].values)
rows = []
for t_idx, t in enumerate(times):
    # find nearest best-track row
    i = int(np.argmin(np.abs(bt["time"].values - np.datetime64(t))))
    lat0, lon0 = bt.loc[i, "lat"], bt.loc[i, "lon"]
    vmax, mslp = bt.loc[i, "vmax"], bt.loc[i, "mslp"]

    # radial bins
    LAT, LON = np.meshgrid(lat, lon, indexing="ij")
    RKM = haversine_km(LAT, LON, lat0, lon0)

    # masks per ring
    stats = {}
    for r_lo, r_hi in zip(R_BINS_KM[:-1], R_BINS_KM[1:]):
        mask = (RKM >= r_lo) & (RKM < r_hi)
        if mask.sum() < 50:
            continue
        kappa_m = np.nanmean(kappa_field[t_idx,:,:][mask])
        g2_m    = np.nanmean(g2_field[t_idx,:,:][mask])
        # distribute surface gamma_phi to lowest ring (or area-avg)
        gamma_m = np.nanmean(gamma_phi_surf.sel(time=t).values[mask]) if "time" in gamma_phi_surf.dims else np.nanmean(gamma_phi_surf.values[mask])
        I = g2_m / ( (kappa_m + 1e-12) * (gamma_m + 1e-12) )
        stats[(r_lo,r_hi)] = (kappa_m, gamma_m, g2_m, I)

    # take inner-ring summary for a row (or area-weighted combination)
    if stats:
        (kappa_m, gamma_m, g2_m, I_m) = list(stats.values())[0]
        rows.append(dict(time=t, vmax=vmax, mslp=mslp,
                         kappa=kappa_m, gamma_phi=gamma_m, g2=g2_m, I_GQR=I_m))

df = pd.DataFrame(rows).dropna()

# --------------------------
# 4) Outputs
# --------------------------
print(df.head())
df.to_csv("storm_GQR_summary.csv", index=False)

plt.figure()
plt.plot(df["time"], df["I_GQR"], label="I_GQR (inner ring)")
plt.ylabel("Shield index (arb.)"); plt.xlabel("Time"); plt.legend(); plt.tight_layout()
plt.savefig("storm_I_GQR_timeseries.png", dpi=200)

plt.figure()
plt.scatter(df["I_GQR"], df["vmax"])
plt.xlabel("I_GQR"); plt.ylabel("Max wind (kt)")
plt.tight_layout(); plt.savefig("I_GQR_vs_vmax.png", dpi=200)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/era5_storm_cube.nc', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)